In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap

In [2]:
news_coords = pd.read_csv('../tagnews/data/homicide-2015-lat-longs.csv')
news_coords = news_coords[['lat', 'long']].values

In [3]:
crimes = pd.read_csv('../tagnews/data/Crimes_-_2001_to_present.csv')

In [4]:
crimes.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

In [5]:
crimes = crimes.loc[crimes['Primary Type'] == 'HOMICIDE', :]
times = pd.to_datetime(crimes['Date'])
crimes = crimes.loc[('2015' <= times) & (times <= '2016')]
# crime_coords = crimes[['Latitude', 'Longitude']]
# crime_coords = crime_coords.loc[~pd.isnull(crime_coords).any(axis=1)]
cavc = crimes['Community Area'].value_counts()
com_area = pd.DataFrame(np.array([cavc.index, cavc.values.flatten()]).T, columns=['area', 'count'])
com_area['area'] = com_area['area'].astype(int)
com_area['area'] = com_area['area'].astype(str)
com_area.set_index('area', drop=False, inplace=True)

In [6]:
import json
from collections import defaultdict
from shapely.geometry import shape, Point
# depending on your version, use: from shapely.geometry import shape, Point

# load GeoJSON file containing sectors
with open('com-areas.geojson') as f:
    js = json.load(f)

counts = defaultdict(lambda : 0)

for point in news_coords:
    point = Point([point[1], point[0]])
    for feature in js['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            counts[feature['properties']['area_num_1']] += 1

In [7]:
com_area['actual_proportions'] = com_area['count'] / com_area['count'].sum()
com_area['news_counts'] = pd.Series(dict(counts))
com_area['news_proportions'] = com_area['news_counts'] / com_area['news_counts'].sum()
com_area['news_minus_actual'] = com_area['news_proportions'] - com_area['actual_proportions']

In [10]:
m = folium.Map(location=[41.84871, -87.6298],
               zoom_start=11,
               tiles='cartodbpositron')

m.choropleth(geo_data='../tagnews/data/Boundaries - Community Areas (current).geojson',
             name='homicide',
             data=com_area,
             columns=['area', 'news_minus_actual'],
             key_on='feature.properties.area_num_1',
             threshold_scale=[-.3, -.1, 0, .1, .3],
             fill_color='PRGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='News under reporting (-) vs. over reporting (+) ')
folium.LayerControl().add_to(m)

# for coord in np.unique(news_coords, axis=0):
#     folium.Marker(coord, icon=folium.Icon(color='gray', icon='newspaper', prefix='fa')).add_to(m)
# for coord in np.unique(crime_coords, axis=0):
#     folium.Marker(coord, icon=folium.Icon(color='blue', icon='gavel', prefix='fa')).add_to(m)

# HeatMap(np.unique(news_coords, axis=0).tolist()).add_to(m)

# folium.GeoJson(
#     'com-areas.geojson',
#     name='geojson'
# ).add_to(m)

m.save('homicide-map.html')

In [12]:
m = folium.Map(location=[41.84871, -87.6298],
               zoom_start=11,
               tiles='cartodbpositron')

m.choropleth(geo_data='../tagnews/data/Boundaries - Community Areas (current).geojson',
             name='homicide',
             data=com_area,
             columns=['area', 'news_minus_actual'],
             key_on='feature.properties.area_num_1',
             threshold_scale=[-.3, -.1, 0, .1, .3],
             fill_color='PRGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='News under reporting (-) vs. over reporting (+) ')
folium.LayerControl().add_to(m)

# for coord in np.unique(news_coords, axis=0):
#     folium.Marker(coord, icon=folium.Icon(color='gray', icon='newspaper', prefix='fa')).add_to(m)
# for coord in np.unique(crime_coords, axis=0):
#     folium.Marker(coord, icon=folium.Icon(color='blue', icon='gavel', prefix='fa')).add_to(m)

# HeatMap(np.unique(news_coords, axis=0).tolist()).add_to(m)

# folium.GeoJson(
#     'com-areas.geojson',
#     name='geojson'
# ).add_to(m)

m.save('homicide-map.html')